### Seleccionamos este dataSet para que se pueda calcular los valores de los jugadores en el mercado


In [624]:
import pandas as pd
import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer


In [625]:
df=pd.read_csv("estado-animo.csv")
df.head(2)

,ID,Time,Age,HRV (ms),GSR (μS),EEG Power Bands,Blood Pressure (mmHg),Oxygen Saturation (%),Heart Rate (BPM),Ambient Noise (dB),Cognitive Load,Mood State,Psychological State,Respiration Rate (BPM),Skin Temp (°C),Focus Duration (s),Task Type,Gender
0,1,2024-01-01 00:00:00,22,3.303.973.885.213.790,1.031.805.777.942.350,"[0.7583653347946298, 1.423247998317594, 0.6157...",114/79,9.843.331.161.949.940,98,56.863.054.434.594,Low,Anxious,Stressed,21,3.456.648.362.706.430,27,Exam,Female
1,2,2024-01-01 00:00:01,23,49.914.651.237.079.800,13.409.828.672.616.600,"[0.5520419333516282, 1.858065835142619, 0.3766...",113/86,9.894.450.488.549.340,70,4.534.342.962.969.860,Low,Neutral,Stressed,21,3.535.859.349.215.910,282,Assignment,Male


In [626]:
traduccion_estado_animo = {
    'ID': 'ID',
    'Time': 'Tiempo',
    'Age': 'Edad',
    'HRV (ms)': 'VFC_ms',  # Variabilidad de Frecuencia Cardíaca en milisegundos
    'GSR (μS)': 'RGP_microS',  # Respuesta Galvánica de la Piel en microSiemens
    'EEG Power Bands': 'Bandas_Poder_EEG',
    'Blood Pressure (mmHg)': 'Presion_Arterial_mmHg',
    'Oxygen Saturation (%)': 'Saturacion_Oxigeno_porcentaje',
    'Heart Rate (BPM)': 'Frecuencia_Cardiaca_BPM',
    'Ambient Noise (dB)': 'Ruido_Ambiental_dB',
    'Cognitive Load': 'Carga_Cognitiva',
    'Mood State': 'Estado_Animo',
    'Psychological State': 'Estado_Psicologico',
    'Respiration Rate (BPM)': 'Frecuencia_Respiratoria_BPM',
    'Skin Temp (°C)': 'Temperatura_Piel_Celsius',
    'Focus Duration (s)': 'Duracion_Enfoque_segundos',
    'Task Type': 'Tipo_Tarea',
    'Gender': 'Genero'
}

# Aplicar la traducción a tu dataset
df_renombrado_animo = df.rename(columns=traduccion_estado_animo)
df_renombrado_animo.head(2)

,ID,Tiempo,Edad,VFC_ms,RGP_microS,Bandas_Poder_EEG,Presion_Arterial_mmHg,Saturacion_Oxigeno_porcentaje,Frecuencia_Cardiaca_BPM,Ruido_Ambiental_dB,Carga_Cognitiva,Estado_Animo,Estado_Psicologico,Frecuencia_Respiratoria_BPM,Temperatura_Piel_Celsius,Duracion_Enfoque_segundos,Tipo_Tarea,Genero
0,1,2024-01-01 00:00:00,22,3.303.973.885.213.790,1.031.805.777.942.350,"[0.7583653347946298, 1.423247998317594, 0.6157...",114/79,9.843.331.161.949.940,98,56.863.054.434.594,Low,Anxious,Stressed,21,3.456.648.362.706.430,27,Exam,Female
1,2,2024-01-01 00:00:01,23,49.914.651.237.079.800,13.409.828.672.616.600,"[0.5520419333516282, 1.858065835142619, 0.3766...",113/86,9.894.450.488.549.340,70,4.534.342.962.969.860,Low,Neutral,Stressed,21,3.535.859.349.215.910,282,Assignment,Male


In [627]:
df_renombrado_animo[['presion_sistolica', 'presion_diastolica']] = df_renombrado_animo['Presion_Arterial_mmHg'].str.split('/', expand=True).astype(int)
df_renombrado_animo.head(2)



,ID,Tiempo,Edad,VFC_ms,RGP_microS,Bandas_Poder_EEG,Presion_Arterial_mmHg,Saturacion_Oxigeno_porcentaje,Frecuencia_Cardiaca_BPM,Ruido_Ambiental_dB,Carga_Cognitiva,Estado_Animo,Estado_Psicologico,Frecuencia_Respiratoria_BPM,Temperatura_Piel_Celsius,Duracion_Enfoque_segundos,Tipo_Tarea,Genero,presion_sistolica,presion_diastolica
0,1,2024-01-01 00:00:00,22,3.303.973.885.213.790,1.031.805.777.942.350,"[0.7583653347946298, 1.423247998317594, 0.6157...",114/79,9.843.331.161.949.940,98,56.863.054.434.594,Low,Anxious,Stressed,21,3.456.648.362.706.430,27,Exam,Female,114,79
1,2,2024-01-01 00:00:01,23,49.914.651.237.079.800,13.409.828.672.616.600,"[0.5520419333516282, 1.858065835142619, 0.3766...",113/86,9.894.450.488.549.340,70,4.534.342.962.969.860,Low,Neutral,Stressed,21,3.535.859.349.215.910,282,Assignment,Male,113,86


In [628]:
df_renombrado_animo["label"]=df_renombrado_animo["Estado_Psicologico"].map({"Stressed":1, "Relaxed":0, "Focused":2, "Anxious":3})
df_renombrado_animo["Genero"]=df_renombrado_animo["Genero"].map({"Male":1, "Female":0, "Other":2} )

In [629]:
df_renombrado_animo["Estado_Psicologico"].value_counts()

Estado_Psicologico
Stressed    263
Focused     249
Relaxed     247
Anxious     241
Name: count, dtype: int64

In [630]:
df_renombrado_animo.head(2)

,ID,Tiempo,Edad,VFC_ms,RGP_microS,Bandas_Poder_EEG,Presion_Arterial_mmHg,Saturacion_Oxigeno_porcentaje,Frecuencia_Cardiaca_BPM,Ruido_Ambiental_dB,...,Estado_Animo,Estado_Psicologico,Frecuencia_Respiratoria_BPM,Temperatura_Piel_Celsius,Duracion_Enfoque_segundos,Tipo_Tarea,Genero,presion_sistolica,presion_diastolica,label
0,1,2024-01-01 00:00:00,22,3.303.973.885.213.790,1.031.805.777.942.350,"[0.7583653347946298, 1.423247998317594, 0.6157...",114/79,9.843.331.161.949.940,98,56.863.054.434.594,...,Anxious,Stressed,21,3.456.648.362.706.430,27,Exam,0,114,79,1
1,2,2024-01-01 00:00:01,23,49.914.651.237.079.800,13.409.828.672.616.600,"[0.5520419333516282, 1.858065835142619, 0.3766...",113/86,9.894.450.488.549.340,70,4.534.342.962.969.860,...,Neutral,Stressed,21,3.535.859.349.215.910,282,Assignment,1,113,86,1


In [631]:
x_train=df_renombrado_animo[["Edad","presion_sistolica", "presion_diastolica", "Frecuencia_Cardiaca_BPM", "Duracion_Enfoque_segundos", "Genero"]]
y_train=df_renombrado_animo["label"]

In [632]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=5000)

In [633]:
model.fit(x_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,5000
,multi_class,'deprecated'


In [634]:
df_renombrado_animo.head(1)



,ID,Tiempo,Edad,VFC_ms,RGP_microS,Bandas_Poder_EEG,Presion_Arterial_mmHg,Saturacion_Oxigeno_porcentaje,Frecuencia_Cardiaca_BPM,Ruido_Ambiental_dB,...,Estado_Animo,Estado_Psicologico,Frecuencia_Respiratoria_BPM,Temperatura_Piel_Celsius,Duracion_Enfoque_segundos,Tipo_Tarea,Genero,presion_sistolica,presion_diastolica,label
0,1,2024-01-01 00:00:00,22,3.303.973.885.213.790,1.031.805.777.942.350,"[0.7583653347946298, 1.423247998317594, 0.6157...",114/79,9.843.331.161.949.940,98,56.863.054.434.594,...,Anxious,Stressed,21,3.456.648.362.706.430,27,Exam,0,114,79,1


In [635]:
prediccion=model.predict([[22,114, 79, 98, 27, 0
                           ]])
if prediccion==0:
        print("Relaxed")
elif prediccion==1:
        print("Stressed")
elif prediccion==2:
        print("Focused")
elif prediccion==3:
        print("Anxio")
print(prediccion)

Anxio
[3]


c:\Users\Emilio\OneDrive\Desktop\trabajoIntegrador\env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
